In [1]:
import pickle5 as pickle
import os
from transformers import AutoTokenizer
from tqdm.auto import tqdm
from collections import Counter
from transformers import pipeline
import pandas as pd

stanza_ents_file_path = os.path.abspath('../../data/stanza_ents-from_questions.pkl')
stanza_ents_main = pickle.load(open(stanza_ents_file_path, 'rb'))

2023-01-16 14:45:14.633957: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-16 14:45:14.633978: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from collections import Counter
s = Counter(stanza_ents_main)

In [5]:
s = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [6]:
s.get_vocab()

{'##pone': 29513,
 'proper': 5372,
 '##同': 30320,
 'properly': 7919,
 'laureate': 17656,
 '##ւ': 29785,
 '##poulos': 24662,
 '##sson': 7092,
 'statues': 11342,
 'sells': 15187,
 '##nl': 20554,
 'vertically': 20018,
 'oak': 6116,
 'faults': 19399,
 'pike': 12694,
 '##joy': 24793,
 'gills': 29468,
 'shan': 17137,
 'orbits': 20347,
 'partnership': 5386,
 '##ನ': 29936,
 'contractual': 27948,
 'divorce': 8179,
 '##rm': 10867,
 'contemplated': 23133,
 '##ppy': 27659,
 'diagnostic': 16474,
 'photography': 5855,
 '##he': 5369,
 'allotted': 23932,
 'web': 4773,
 'media': 2865,
 'outreach': 15641,
 'detective': 6317,
 'proceeding': 18207,
 'dwellers': 28857,
 '##খ': 29890,
 'newest': 14751,
 '##uri': 9496,
 'dislike': 18959,
 '[unused870]': 875,
 'attempt': 3535,
 'hunter': 4477,
 '##sford': 17658,
 '##shire': 7430,
 'alexei': 21219,
 '##bat': 14479,
 '##wes': 18192,
 'sporting': 7419,
 '幸': 1841,
 '##gg': 13871,
 'greensboro': 27905,
 'brute': 26128,
 'pepsi': 27237,
 'obscene': 27744,
 'conclu

In [3]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model_vocab = list(tokenizer.vocab.keys())

#Keeping only unique enitities
#stanza_ents_main = list(set(stanza_ents_main))

#Removing those entities that are also in the model vocab since we don't want 2 of the same token embedding.
ent_in_model_vocab = []
for ent in tqdm(stanza_ents_main):
    if ent in model_vocab:
        ent_in_model_vocab.append(ent)

for ent in ent_in_model_vocab:
    stanza_ents_main.remove(ent)

  0%|          | 0/2880 [00:00<?, ?it/s]

In [ ]:
#Removing 1 length entities - I realized that this doesn't really make much of a difference.
'''
keys_to_remove = []
for key in stanza_ents_main:
    if len(key) == 1:
        keys_to_remove.append(key)
        
for key in keys_to_remove:
    stanza_ents_main.remove(key)
'''

In [ ]:
#Setting up the text-generator
generator = pipeline('text-generation', model = "bigscience/bloom-560m", device=0)

In [ ]:
sample_id = []
title = []
contexts = []
questions = []
answers = []

def triple_gen(ent, ques):
    questions.append(ques)
    ctx_body = generator(ent, renormalize_logits=True, 
                              do_sample=True, max_new_tokens=2000)[0]['generated_text']
    
    ans_prefix = ques[ques.find(',')+1: ques.find('?')].strip()
    ans_text = generator(ans_prefix, renormalize_logits=True, do_sample=True,
                         max_new_tokens=100)[0]['generated_text']
    
    total_context = ctx_body + '' + ans_text
    contexts.append(total_context)
    
    answers.append({'text': [ans_text], 'answer_start': [total_context.find(ans_text)]})

c = 0
for ent in tqdm(stanza_ents_main):
    #Question-1 (Is)
    sample_id.append(str(c))
    c += 1
    title.append('Is' + ent)
    triple_gen(ent, generator(f"Question: Is {ent},", renormalize_logits=True, do_sample=True, 
                               forced_eos_token_id=[34], 
                               max_new_tokens=10)[0]['generated_text'].strip('Question: '))
    
    #Question-2 (In)
    sample_id.append(str(c))
    c += 1
    title.append('In' + ent)
    triple_gen(ent, generator(f"Question: In {ent},", renormalize_logits=True, do_sample=True, 
                               forced_eos_token_id=[34], 
                               max_new_tokens=10)[0]['generated_text'].strip('Question: '))
    
    #Question-3 (Why)
    sample_id.append(str(c))
    c += 1
    title.append('Why' + ent)
    triple_gen(ent, generator(f"Question: Why {ent},", renormalize_logits=True, do_sample=True, 
                               forced_eos_token_id=[34], 
                               max_new_tokens=10)[0]['generated_text'].strip('Question: '))
    
    #Question-4 (During)
    sample_id.append(str(c))
    c += 1
    title.append('During' + ent)
    triple_gen(ent, generator(f"Question: During {ent},", renormalize_logits=True, do_sample=True, 
                               forced_eos_token_id=[34], 
                               max_new_tokens=10)[0]['generated_text'].strip('Question: '))
    
    #Question-5 (Which)
    sample_id.append(str(c))
    c += 1
    title.append('Which' + ent)
    triple_gen(ent, generator(f"Question: Which {ent},", renormalize_logits=True, do_sample=True, 
                               forced_eos_token_id=[34], 
                               max_new_tokens=10)[0]['generated_text'].strip('Question: '))
    
#Saving the dataframe as parquet since it was messing up formats.
pd.DataFrame(zip(sample_id, title, context, question, answers), 
             columns = ['id', 'title', 'context', 'question'
                       ,'answers']).to_parquet('mini-corpus-for-extended-QA')